In [24]:
import csv

reviews=[]
for row in csv.reader(open('C:\\Users\\Hassan\\Desktop\\bbcurdu.csv', encoding = "utf-8")):
    reviews.append(row)
reviews = reviews[1:]  #This command removes the first row of the file as it only contains label(Heading)
print(reviews)


[['کراچی پولیس کے ترجمان کی جانب سے جاری اعلامیے کے مطابق نیشنل اسٹیڈیم سمیت تمام متعلقہ روٹس اور ہوٹلوں پر پولیس کی بھاری نفری اور کمانڈوزکھلاڑیوں کی سکیورٹی کے لیے موجود رہیں گے، کراچی پولیس کے 24 سینئر افسران سمیت 59 ڈی ایس پیز، 474 نان گزیٹڈ افسران،4275 ہیڈکانسٹیبل ، کانسٹیبل ، ریپڈ رسپانس فورس کے 345 جوان اور اسپیشل سیکیورٹی یونٹ کے 1018 کمانڈوز تعینات کیے گئے ہیں۔نیشنل اسٹیڈیم کراچی اور ہوٹلوں پر اسپیشل برانچ کے سادہ لباس اہلکاروں کی بڑی تعداد بھی سکیورٹی کے فرائض سر انجام دے رہی ہے غیر معمولی حالات سے نمٹنے کے لیے اسپیشل سیکیورٹی یونٹ کی SWAT ٹیمیں ہمہ وقت فوری رسپانس کے لیے تیار رہیں گی۔کراچی پولیس کے ترجمان نے کہا کہ ٹریفک پولیس نے پی ایس ایل فائیو میں شرکت کے لیے آنے والے ملکی اور غیر ملکی کھلاڑیوں کی نیشنل اسٹیڈیم کی طرف آمدورفت کے اوقات میں شہریوں کی سہولت کے لیے متبادل روٹ پلان پہلے ہی جاری کردیا ہے، کراچی میں امن کی بحالی کے لیے پولیس نے بڑی قربانیاں دی ہیں، انٹرنیشنل کھلاڑیوں کی آمد شہر میں امن و امان کے قیام کی دلیل ہے، کراچی پولیس عوام کے تحفظ اور قانون کی بالادستی 

c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
import spacy
from spacy.lang.ur import Urdu
parser = Urdu()
from spacy.lang.ur.stop_words import STOP_WORDS
import nltk
stopwords = set(nltk.corpus.stopwords.words('urdu'))
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

final=[]     #In this list all of our pre processed data will be stored
for line in reviews:  #Break sentence into list words
    for words in line:
        tokens = tokenize(words)
        tokens = [token for token in tokens if len(token) > 3]
        tokens = [token for token in tokens if token not in stopwords]
        tokens = [get_lemma(token) for token in tokens]
        final.append(tokens)
print(final)

c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['کراچی', 'پولیس', 'ترجمان', 'جانب', 'جاری', 'اعلامیے', 'مطابق', 'نیشنل', 'اسٹیڈیم', 'سمیت', 'متعلقہ', 'روٹس', 'ہوٹلوں', 'پولیس', 'بھاری', 'نفری', 'کمانڈوزکھلاڑیوں', 'سکیورٹی', 'موجود', 'کراچی', 'پولیس', 'سینئر', 'افسران', 'سمیت', 'گزیٹڈ', 'افسران،4275', 'ہیڈکانسٹیبل', 'کانسٹیبل', 'ریپڈ', 'رسپانس', 'فورس', 'جوان', 'اسپیشل', 'سیکیورٹی', 'یونٹ', '1018', 'کمانڈوز', 'تعینات', 'ہیں۔نیشنل', 'اسٹیڈیم', 'کراچی', 'ہوٹلوں', 'اسپیشل', 'برانچ', 'سادہ', 'لباس', 'اہلکاروں', 'تعداد', 'سکیورٹی', 'فرائض', 'انجام', 'معمولی', 'حالات', 'نمٹنے', 'اسپیشل', 'سیکیورٹی', 'یونٹ', 'swat', 'ٹیمیں', 'فوری', 'رسپانس', 'تیار', 'گی۔کراچی', 'پولیس', 'ترجمان', 'ٹریفک', 'پولیس', 'فائیو', 'شرکت', 'آنے', 'ملکی', 'ملکی', 'کھلاڑیوں', 'نیشنل', 'اسٹیڈیم', 'آمدورفت', 'اوقات', 'شہریوں', 'سہولت', 'متبادل', 'پلان', 'جاری', 'کراچی', 'بحالی', 'پولیس', 'قربانیاں', 'انٹرنیشنل', 'کھلاڑیوں', 'آمد', 'امان', 'قیام', 'دلیل', 'کراچی', 'پولیس', 'عوام', 'تحفظ', 'قانون', 'بالادستی', 'ممکن', 'اقدامات'], ['مارچ', 'مرحلہ', 'مکمل', 'میچز', 'ک

In [26]:
from gensim import corpora
dictionary = corpora.Dictionary(final)
print(dictionary)

Dictionary(1444 unique tokens: ['1018', 'swat', 'اسٹیڈیم', 'اسپیشل', 'اعلامیے']...)


c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
corpus = [dictionary.doc2bow(text) for text in final]
print(corpus)

[[(0, 1), (1, 1), (2, 3), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 2), (35, 2), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 1), (62, 1), (63, 7), (64, 1), (65, 5), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 2)], [(2, 3), (13, 1), (26, 2), (31, 1), (34, 1), (38, 1), (44, 1), (46, 1), (56, 2), (58, 1), (59, 2), (62, 1), (63, 1), (65, 2), (66, 1), (68, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 3), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1),

c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)
ldamodel.show_topics()  

c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.008*"کراچی" + 0.008*"پولیس" + 0.006*"اسٹیڈیم" + 0.005*"جونی" + 0.005*"لباس" + 0.005*"کیلیے" + 0.004*"جاری" + 0.004*"زلمی" + 0.004*"جانب" + 0.004*"کھلاڑیوں"'),
 (1,
  '0.007*"کردار" + 0.006*"ڈرامے" + 0.005*"ڈسپلے" + 0.004*"کیلیے" + 0.004*"کنگز" + 0.004*"مرحلے" + 0.004*"ڈراما" + 0.004*"حاصل" + 0.003*"کارکردگی" + 0.003*"بابر"'),
 (2,
  '0.008*"کردار" + 0.008*"ڈرامے" + 0.005*"بھارتی" + 0.003*"جوان" + 0.003*"جہاں" + 0.003*"پاکستانی" + 0.003*"آپشن" + 0.003*"جاری" + 0.003*"مطابق" + 0.003*"تنقید"'),
 (3,
  '0.008*"آپشن" + 0.006*"میچز" + 0.006*"پاکستان" + 0.005*"عباسی" + 0.005*"مطابق" + 0.005*"بھارتی" + 0.005*"ضرورت" + 0.005*"مواد" + 0.004*"دوران" + 0.004*"شامل"'),
 (4,
  '0.006*"کارکردگی" + 0.005*"میچز" + 0.005*"اسٹوریج" + 0.005*"کیلیے" + 0.004*"مرحلے" + 0.004*"کرکٹر" + 0.004*"کرکٹ" + 0.004*"آئی" + 0.004*"گوگل" + 0.004*"مطابق"')]

In [36]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.066045 -0.002921       1        1  27.164480
3     -0.051119  0.021956       2        1  25.373674
1      0.010213 -0.008796       3        1  25.157956
4     -0.026030 -0.052532       4        1  12.722035
2      0.000892  0.042294       5        1   9.581855, topic_info=          Term      Freq      Total Category  logprob  loglift
891      ڈرامے  8.000000   8.000000  Default  30.0000  30.0000
857      کردار  8.000000   8.000000  Default  29.0000  29.0000
718      آپشن  6.000000   6.000000  Default  28.0000  28.0000
63       پولیس  6.000000   6.000000  Default  27.0000  27.0000
1245   اسٹوریج  4.000000   4.000000  Default  26.0000  26.0000
...        ...       ...        ...      ...      ...      ...
136       حاصل  0.657940   6.439193   Topic5  -5.8950   0.0643
2      اسٹیڈیم  0.670429   7.120420   Topic5  -5.8762  -0.0175
65       کراچی  0.670620  10.834536   Topic5  -5.8759  -0.4370
256   کارکردگی  0.607295   7.114696   Topic5  -5.9751  -0.1156
68    کھلاڑیوں  0.606151   7.364607   Topic5  -5.9770  -0.1520

[320 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
924       3  0.680929       2014
1018      1  0.897726      اخبار
710       1  0.253066     اداکار
710       2  0.253066     اداکار
710       3  0.506132     اداکار
...     ...       ...        ...
1339      3  0.569686  ہولوگرافک
71        1  0.474732     ہوٹلوں
1072      1  0.702455  ہوگا۔واضح
1443      4  0.619192     ہے۔اسی
74        1  0.483939       یونٹ

[396 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 5, 3])

In [30]:
!python -m pip install -U pyLDAvis


c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Requirement already up-to-date: pyLDAvis in c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages (2.1.2)


You should consider upgrading via the 'C:\Users\Hassan\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.
